You may find this series of notebooks at [databricks-solutions/realtime-rag-agents-databricks-youcom](https://github.com/databricks-solutions/realtime-rag-agents-databricks-youcom). For more information about this solution accelerator, visit the [blog post](https://you.com/articles/unlocking-real-time-intelligence-for-ai-agents-with-you.com-and-databricks).

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import pyspark.sql.functions as F
import io
import yaml

import mlflow

import mlflow.genai.datasets

import requests

from agent import AGENT

from mlflow.genai.scorers import (
    RetrievalGroundedness,
    RelevanceToQuery,
    Safety,
    Guidelines,
    Correctness
)

# Create Agent

1. Use the preivous configurations to stream responses from the defined agent.

2. Create a seed dataset that will be evaluated in `04-evaluate-agent`


## 0. Load Configurations from `01-agent-configs`

In [0]:
# load the configuration settings from earlier
cfg = yaml.safe_load(open("common.yaml"))

catalog = cfg.get("catalog", "main")
schema = cfg.get("schema", "default")

uc_schema = f"{catalog}.{schema}"
# This table will be created in the above UC schema
evaluation_dataset_table_name = "test_evaluation_set"

evaluation_dataset_path = f"{catalog}.{schema}.{evaluation_dataset_table_name}"

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
experiment_name = notebook_path + "_exp"

mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_name)

In [0]:
def create_message_payload(question):
  return {"messages": [{"role": "user", "content": question}]}

## 1. Stream Agent's Response

In [0]:
question = "What is the capital of France?"
message = create_message_payload(question)

resp = AGENT.predict(message)
print(resp)

In [0]:
# stream the Assistant's response
for chunk in AGENT.predict_stream(message):
  if chunk.delta.role == "assistant":
    print(chunk.delta.content)

## 2. Seed Examples for Evaluation Dataset

Use the AGENT to predict a couple example questions. Then use the Traces to seed an Evaluation Dataset.

In [0]:
questions = ["What is the capital of Germany?", 
             "Who won the Women's 800m Freestyle at the most recent World Swimming Championships?", ]

run_id = None
with mlflow.start_run() as run:
  for q in questions:
    message = create_message_payload(q)
    resp = AGENT.predict(message)

  run_id = run.info.run_id